In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib

In [5]:
import sys
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
#import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
# List of Data Sources
# 1. Chicago Crime Database --> https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
# 2. Chicago IUCR Code Mapping --> https://data.cityofchicago.org/Public-Safety/Chicago-Police-Department-Illinois-Uniform-Crime-R/c7ck-438e
# 3. Hate Crime DataBase --> https://home.chicagopolice.org/statistics-data/data-dashboards/hate-crime-dashboard/
# 4. ESRI Latitude Longitude Mapping to Zip Code --> https://www.esri.com/en-us/home
# 5. Output from Recommendation Engine --> ten_user_recommendations.csv

In [6]:
dfCrime = pd.read_csv("data/crimes_2015_to_present_with_feature.csv")
print(dfCrime.shape)
dfCrime.head()

C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (20,21,22,23,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1677352, 32)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620


In [7]:
dfCrime.columns

Index(['DateFormatted', 'ID', 'Case_Number', 'IUCR', 'Primary_Type',
       'Description', 'Location_Description', 'Arrest', 'Domestic', 'FBI_Code',
       'X_Coordinate', 'Y_Coordinate', 'Year', 'Latitude', 'Longitude',
       'Location', 'isDay', 'hour', 'IndexCode', 'isSerious',
       'HATE_CRIME_DISPOSITION', 'HATE_CRIME_BIAS', 'HATE_CRIME_BIAS_CLASS',
       'HATE_CRIME_BIAS_GROUP', 'HATE_CRIME_Location_Name',
       'HATE_CRIME_Location_Code', 'HATE_CRIME_TYPE', 'isHateCrime',
       'isHateCrimeDisposition', 'location_group', 'IsResidentialProperty',
       'zip'],
      dtype='object')

In [8]:
dfFilteredCopy = dfCrime.copy()

In [9]:
print(dfFilteredCopy.Latitude.isna().sum())
print(dfFilteredCopy.Longitude.isna().sum())
dfFilteredCopy.dropna(subset=['Latitude','Longitude'],inplace=True)
print(dfFilteredCopy.Latitude.isna().sum())
print(dfFilteredCopy.Longitude.isna().sum())

0
0
0
0


In [10]:
import geopy
from geopy.geocoders import Nominatim

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    zipVal = location.raw['address'].get('postcode','99999')
    if zipVal:
        df['zipcode'] = zipVal
    return df



In [11]:
# debug
data = [[41.815117282,-87.669999562,(41.815117282,-87.669999562)],
        [41.895080471, -87.765400451, (41.895080471, -87.765400451)]] 
                                                                    
dfTest = pd.DataFrame(data,columns=['Latitude','Longitude','Location'])
print(dfTest.shape)
dfTest.head()

(2, 3)


,Latitude,Longitude,Location
0,41.815117,-87.6700,"(41.815117282, -87.669999562)"
1,41.895080,-87.7654,"(41.895080471, -87.765400451)"


In [12]:
dfFilteredCopy['geom'] = dfFilteredCopy['Latitude'].map(str) + ',' + dfFilteredCopy['Longitude'].map(str)
dfFilteredCopy['geom'][0]

'41.815117282,-87.669999562'

In [13]:
# This did not work -- 
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#geolocator = geopy.Nominatim(user_agent='crimedb-application1',  timeout=10)
#rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.001)

In [39]:
def get_zipcodev2(lat,lon):
    #print(loc)
    location = geolocator.reverse((lat,lon))
    zipVal = location.raw['address'].get('postcode','99999')
    return zipVal

In [15]:
geolocator = geopy.Nominatim(user_agent='crimedb-application2')

In [16]:

dfTest1 = dfTest.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')
print(dfTest1.shape)
dfTest1.head()

(2, 4)


,Latitude,Longitude,Location,zipcode
0,41.815117,-87.6700,"(41.815117282, -87.669999562)",60636
1,41.895080,-87.7654,"(41.895080471, -87.765400451)",60630


In [17]:
dfFilteredCopy['Latitude'] = dfFilteredCopy['Latitude'].astype(float)
dfFilteredCopy['Longitude'] = dfFilteredCopy['Longitude'].astype(float)

In [19]:
dfFilteredCopy.columns

Index(['DateFormatted', 'ID', 'Case_Number', 'IUCR', 'Primary_Type',
       'Description', 'Location_Description', 'Arrest', 'Domestic', 'FBI_Code',
       'X_Coordinate', 'Y_Coordinate', 'Year', 'Latitude', 'Longitude',
       'Location', 'isDay', 'hour', 'IndexCode', 'isSerious',
       'HATE_CRIME_DISPOSITION', 'HATE_CRIME_BIAS', 'HATE_CRIME_BIAS_CLASS',
       'HATE_CRIME_BIAS_GROUP', 'HATE_CRIME_Location_Name',
       'HATE_CRIME_Location_Code', 'HATE_CRIME_TYPE', 'isHateCrime',
       'isHateCrimeDisposition', 'location_group', 'IsResidentialProperty',
       'zip', 'geom'],
      dtype='object')

In [20]:
print(dfFilteredCopy.shape)
dfFilteredCopy.head()

(1677352, 33)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip,geom
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,"41.815117282,-87.669999562"
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651,"41.895080471,-87.765400451"
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618,"41.937405765,-87.716649687"
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644,"41.881903443,-87.755121152"
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620,"41.744378879,-87.658430635"


# Zip code from ESRI DB - based on latitude and longitude (https://www.esri.com/en-us/home)

In [21]:
dfData = pd.read_csv("crimes_2015_to_present_with_zipcodes_enhanced.csv")
print(dfData.shape)
dfData.head()

C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (23,24,25,26,27,29,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1705799, 37)


,OID_,Join_Count,TARGET_FID,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,...,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zipCode,objectid_1,zip
0,1,1,1,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,1.0,60609.0
1,2,1,2,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,...,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,Y,NaN,2.0,60651.0
2,3,1,3,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,...,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,Y,NaN,3.0,60618.0
3,4,1,4,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,4.0,60644.0
4,5,1,5,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,5.0,60620.0


In [22]:
dfDataCopy = dfData.copy()

In [23]:
print(dfData.IsResidentialProperty.value_counts())


Y    1066674
N     632069
Name: IsResidentialProperty, dtype: int64


In [24]:
dfData['IsResidentialProperty'] = dfData['IsResidentialProperty'].replace({'Y':'N','N':'Y'})
print(dfData.IsResidentialProperty.value_counts())

N    1066674
Y     632069
Name: IsResidentialProperty, dtype: int64


In [25]:
display(dfData.head())

,OID_,Join_Count,TARGET_FID,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,...,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zipCode,objectid_1,zip
0,1,1,1,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,NaN,1.0,60609.0
1,2,1,2,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,...,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,NaN,2.0,60651.0
2,3,1,3,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,...,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,NaN,3.0,60618.0
3,4,1,4,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,NaN,4.0,60644.0
4,5,1,5,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,NaN,5.0,60620.0


In [26]:
targetCols = [ 'DateFormatted', 'ID',
       'Case_Number', 'IUCR', 'Primary_Type', 'Description',
       'Location_Description', 'Arrest', 'Domestic', 'FBI_Code',
       'X_Coordinate', 'Y_Coordinate', 'Year', 'Latitude', 'Longitude',
       'Location', 'isDay', 'hour', 'IndexCode', 'isSerious',
       'HATE_CRIME_DISPOSITION', 'HATE_CRIME_BIAS', 'HATE_CRIME_BIAS_CLASS',
       'HATE_CRIME_BIAS_GROUP', 'HATE_CRIME_Location_Name',
       'HATE_CRIME_Location_Code', 'HATE_CRIME_TYPE', 'isHateCrime',
       'isHateCrimeDisposition', 'location_group', 'IsResidentialProperty',
        'zip']

In [27]:
dfResult = dfData[targetCols]
print(dfResult.shape)
dfResult.head()

(1705799, 32)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609.0
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651.0
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618.0
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644.0
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620.0


In [28]:
dfResult.zip.isna().sum()
dfResult.dropna(subset=['zip'],inplace=True)
dfResult.zip.isna().sum()

C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


0

In [22]:
dfResult['zip']=dfResult['zip'].astype(int)
print(dfResult.shape)
dfResult.head()

(1677352, 32)


C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620


In [23]:
dfResult.to_csv("crimes_2015_to_present_with_feature.csv",index=False)

# Enhance Crime features with mean rating and target crime counts

In [29]:
import pandas as pd

In [30]:
dfResult = pd.read_csv("crimes_2015_to_present_with_feature.csv")
print(dfResult.shape)
dfResult.head()

C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (20,21,22,23,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1677352, 32)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620


In [31]:
primaryTypeList = dfResult.Primary_Type.unique().tolist()
primaryTypeList

['BATTERY',
 'THEFT',
 'NARCOTICS',
 'ASSAULT',
 'BURGLARY',
 'ROBBERY',
 'CRIMINAL DAMAGE',
 'OTHER OFFENSE',
 'WEAPONS VIOLATION',
 'DECEPTIVE PRACTICE',
 'CRIMINAL TRESPASS',
 'MOTOR VEHICLE THEFT',
 'INTERFERENCE WITH PUBLIC OFFICER',
 'OFFENSE INVOLVING CHILDREN',
 'PUBLIC PEACE VIOLATION',
 'GAMBLING',
 'SEX OFFENSE',
 'STALKING',
 'CRIM SEXUAL ASSAULT',
 'PROSTITUTION',
 'KIDNAPPING',
 'CONCEALED CARRY LICENSE VIOLATION',
 'INTIMIDATION',
 'ARSON',
 'NON - CRIMINAL',
 'OBSCENITY',
 'LIQUOR LAW VIOLATION',
 'PUBLIC INDECENCY',
 'NON-CRIMINAL',
 'OTHER NARCOTIC VIOLATION',
 'HUMAN TRAFFICKING',
 'CRIMINAL SEXUAL ASSAULT',
 'HOMICIDE',
 'NON-CRIMINAL (SUBJECT SPECIFIED)',
 'RITUALISM']

In [32]:
targetCrimes = ['CRIM SEXUAL ASSAULT', 'ROBBERY', 'BATTERY','ASSAULT','BURGLARY', 'MOTOR VEHICLE THEFT' , 'THEFT']

In [33]:
dfResult.columns

Index(['DateFormatted', 'ID', 'Case_Number', 'IUCR', 'Primary_Type',
       'Description', 'Location_Description', 'Arrest', 'Domestic', 'FBI_Code',
       'X_Coordinate', 'Y_Coordinate', 'Year', 'Latitude', 'Longitude',
       'Location', 'isDay', 'hour', 'IndexCode', 'isSerious',
       'HATE_CRIME_DISPOSITION', 'HATE_CRIME_BIAS', 'HATE_CRIME_BIAS_CLASS',
       'HATE_CRIME_BIAS_GROUP', 'HATE_CRIME_Location_Name',
       'HATE_CRIME_Location_Code', 'HATE_CRIME_TYPE', 'isHateCrime',
       'isHateCrimeDisposition', 'location_group', 'IsResidentialProperty',
       'zip'],
      dtype='object')

In [34]:
dfFiltered = dfResult[dfResult['Primary_Type'].isin(targetCrimes)]
print(dfFiltered.shape)
dfFiltered.head()

(1041760, 32)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620
5,9/5/2015 13:00:00,10224744,HY411605,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,06,...,NaN,NaN,NaN,NaN,NaN,N,NaN,GROCERY/SUPERMARKET,N,60608


In [35]:
dfGrouped=dfFiltered[['zip','Primary_Type','ID']].\
groupby(['zip','Primary_Type']).count().rename(columns={"ID":'RecoEngine_Count'}).reset_index()
print(dfGrouped.shape)
dfGrouped.head()

(413, 3)


,zip,Primary_Type,RecoEngine_Count
0,60601,ASSAULT,594
1,60601,BATTERY,1366
2,60601,BURGLARY,154
3,60601,CRIM SEXUAL ASSAULT,37
4,60601,MOTOR VEHICLE THEFT,236


In [36]:
dfGrouped1=dfFiltered[['zip','Primary_Type','ID']].\
groupby(['zip']).count().rename(columns={"ID":'RecoEngine_Count'}).reset_index()
print(dfGrouped1.shape)
dfGrouped1.tail(30)

(59, 3)


,zip,Primary_Type,RecoEngine_Count
29,60631,2607,2607
30,60632,19628,19628
31,60633,2228,2228
32,60634,11757,11757
33,60636,29047,29047
34,60637,31151,31151
35,60638,8571,8571
36,60639,24144,24144
37,60640,17519,17519
38,60641,16178,16178


In [37]:
dfResultWithRecoCount = pd.merge(dfResult,dfGrouped,how='left',on=['zip','Primary_Type'])
print(dfResultWithRecoCount.shape)
print(dfResultWithRecoCount.columns)
dfResultWithRecoCount.head()

(1677352, 33)
Index(['DateFormatted', 'ID', 'Case_Number', 'IUCR', 'Primary_Type',
       'Description', 'Location_Description', 'Arrest', 'Domestic', 'FBI_Code',
       'X_Coordinate', 'Y_Coordinate', 'Year', 'Latitude', 'Longitude',
       'Location', 'isDay', 'hour', 'IndexCode', 'isSerious',
       'HATE_CRIME_DISPOSITION', 'HATE_CRIME_BIAS', 'HATE_CRIME_BIAS_CLASS',
       'HATE_CRIME_BIAS_GROUP', 'HATE_CRIME_Location_Name',
       'HATE_CRIME_Location_Code', 'HATE_CRIME_TYPE', 'isHateCrime',
       'isHateCrimeDisposition', 'location_group', 'IsResidentialProperty',
       'zip', 'RecoEngine_Count'],
      dtype='object')


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip,RecoEngine_Count
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,9621.0
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651,7157.0
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618,NaN
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644,5214.0
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620,3627.0


In [38]:
dfResultWithRecoCount['RecoEngine_Count'] = dfResultWithRecoCount['RecoEngine_Count'].fillna(0)
dfResultWithRecoCount['RecoEngine_Count'] = dfResultWithRecoCount['RecoEngine_Count'].astype(int)

In [39]:
print(dfResultWithRecoCount.shape)
dfResultWithRecoCount.head()

(1677352, 33)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip,RecoEngine_Count
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,9621
1,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,...,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,N,60651,7157
2,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,...,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,N,60618,0
3,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60644,5214
4,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,...,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60620,3627


In [40]:
dfUser = pd.read_csv("ten_user_recommendations.csv",encoding='utf-8')
print(dfUser.shape)
dfUser.head()

(590, 10)


,Unnamed: 0,user_index,att,zip,user_rating_x,user_rating_y,user_rating_x.1,user_rating_y.1,user_rating,mean_rating
0,0,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60601,1,4,4,1,4,2.8
1,1,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60602,1,4,4,1,4,2.8
2,2,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60603,1,3,2,1,2,1.8
3,3,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60604,1,4,2,1,2,2.0
4,4,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60605,1,5,5,1,4,3.2


In [41]:
dfUser['mean_rating'] = dfUser['mean_rating'].fillna(0.0)
dfUser.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       590 non-null    int64  
 1   user_index       590 non-null    int64  
 2   att              590 non-null    object 
 3   zip              590 non-null    int64  
 4   user_rating_x    590 non-null    int64  
 5   user_rating_y    590 non-null    int64  
 6   user_rating_x.1  590 non-null    int64  
 7   user_rating_y.1  590 non-null    int64  
 8   user_rating      590 non-null    int64  
 9   mean_rating      590 non-null    float64
dtypes: float64(1), int64(8), object(1)
memory usage: 46.2+ KB


In [42]:
dfUserGroup = dfUser[['user_index','zip','mean_rating']]
dfUserGroup.rename(columns={'mean_rating':'RecoEngine_Rating'},inplace=True)
print(dfUserGroup.shape)
dfUserGroup.head()

(590, 3)


C:\Users\kkcar\Anaconda3\envs\cse6242\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_index,zip,RecoEngine_Rating
0,478,60601,2.8
1,478,60602,2.8
2,478,60603,1.8
3,478,60604,2.0
4,478,60605,3.2


In [38]:
dfUserGroup.to_csv("dfUserGroup.csv",index=False)

In [43]:
dfUserGroup.RecoEngine_Rating.value_counts()

5.0    319
4.8     38
3.4     35
3.8     26
4.2     23
4.4     22
3.6     21
4.6     21
3.2     15
2.8     14
2.0     10
4.0      9
3.0      8
2.6      8
1.8      7
2.2      4
2.4      3
1.0      2
1.6      2
1.2      2
1.4      1
Name: RecoEngine_Rating, dtype: int64

In [44]:
dfFinal = pd.merge(dfResult,dfUserGroup,how='left',on=['zip'])
print(dfFinal.shape)
dfFinal.head()

(16773520, 34)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip,user_index,RecoEngine_Rating
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,478,3.8
1,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,17,5.0
2,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,702,5.0
3,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,111,5.0
4,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,175,5.0


In [45]:
dfFinal['RecoEngine_Rating'] = dfFinal['RecoEngine_Rating'].fillna(0.0)
dfFinal['RecoEngine_Rating'] = dfFinal['RecoEngine_Rating'].astype(float)

print(dfFinal.shape)
dfFinal.head()

(16773520, 34)


,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,...,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zip,user_index,RecoEngine_Rating
0,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,478,3.8
1,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,17,5.0
2,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,702,5.0
3,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,111,5.0
4,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,...,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,Y,60609,175,5.0


In [46]:
dfFinal['RecoEngine_Rating'].value_counts()

5.0    11225994
3.8     1039943
4.8      921537
3.6      790701
3.4      787556
4.6      365350
4.2      351344
4.4      274388
3.2      265944
4.0      226751
2.8      129779
3.0      117506
2.6       82934
2.0       58305
1.8       42229
2.4       31669
2.2       26708
1.0       13437
1.6        9296
1.2        7272
1.4        4877
Name: RecoEngine_Rating, dtype: int64

In [47]:
dfFinal['user_index'].value_counts()

478    1677352
17     1677352
702    1677352
111    1677352
175    1677352
450    1677352
158    1677352
807    1677352
282    1677352
204    1677352
Name: user_index, dtype: int64

In [27]:
dfFinal.to_csv("crimes_2015_to_present_with_reco_engine.csv",index=False)